In [ ]:
from burstfit.fit import BurstFit
from burstfit.data import BurstData
from burstfit.model import Model, SgramModel
from burstfit.utils.plotter import plot_me
from burstfit.utils.functions import pulse_fn_vec, sgram_fn_vec, gauss_norm
from burstfit.io import BurstIO
import logging

In [ ]:
logging_format = "%(asctime)s - %(funcName)s -%(name)s - %(levelname)s - %(message)s"
logging.basicConfig(
    level=logging.INFO,
    format=logging_format,
)

## Burst Fitting

#### Get candidate cutout and preprocess
#### BurstData class (get data cutout and pre-process)

In [ ]:
fil_file = '../tests/data/tcand_2_dm_565.30000_snr_11.92560.fil'

In [ ]:
bd = BurstData(
    fp=fil_file,
    dm=565.3,
    tcand=2,
    width=64,
    snr=11.9,
    min_samp=256,
)

In [ ]:
# We will mask channel numbers from 53 to 64 as they are 0. 
bd.prepare_data(mask_chans=[(53, 64)])

#### Fitting using BurstFit
#### Define relevant functions for profile, spectra and spectrogram

In [ ]:
pnames = ['S', 'mu_t', 'sigma_t', 'tau']
pulseModel = Model(pulse_fn_vec, param_names=pnames)
snames = ['mu_f', 'sigma_f']
spectraModel = Model(gauss_norm, param_names=snames)

In [ ]:
sgramModel = SgramModel(pulseModel, spectraModel, sgram_fn_vec, 
                        mask=bd.mask, clip_fac=bd.clip_fac)

#### Provide basic candidate information to BurstFit: sgram, model, DM, width, RFI mask, etc 

In [ ]:
bf = BurstFit(
    sgram_model=sgramModel,
    sgram=bd.sgram,
    width=bd.width,
    dm=bd.dm,
    foff=bd.foff,
    fch1=bd.fch1,
    tsamp=bd.tsamp,
    clip_fac=bd.clip_fac,
    mask=bd.mask) 

#### Fitting ....

In [ ]:
bf.fitall(plot=False)

### We can use BurstIO class to save the results. 

In [ ]:
bio = BurstIO(bf, bd)

In [ ]:
d = bio.save_results()

#### This saves the relevant parameters: header of the file, candidate parameters, fit results, function names to a JSON file. 
##### Here are the parameters that are saved: 

In [ ]:
d

#### Fitted parameters for all components are saved in sgram_params['all']

In [ ]:
d['sgram_params']['all']

### BurstIO class can also be used to read the json file with fitting results and reproduce the sgram model

In [ ]:
file = '../tests/data/cand_tstart_57645.411052962729_tcand_2.0000000_dm_565.30000_snr_11.90000.json'
bio = BurstIO(jsonfile=file)

#### This reads the json file and sets up relevant functions and metadata

In [ ]:
bio.read_json_and_precalc()

In [ ]:
bio.sgram_params

In [ ]:
# sgram function
bio.sgram_model.sgram_function.__name__

In [ ]:
# profile and spectra function
bio.sgram_model.pulse_model.function.__name__, bio.sgram_model.spectra_model.function.__name__

#### Similar to BurstFit class, we can use bio.model to make the model. This uses the sgram_params or mcmc_params (if available) to make the model. 

In [ ]:
plot_me(bio.model)